In [1]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName('log_reg').getOrCreate()

In [2]:
from pyspark.ml.classification import LogisticRegression

In [3]:
my_data = spark.read.format('libsvm').load('sample_libsvm_data.txt')

In [4]:
my_data.show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
|  0.0|(692,[127,128,129...|
|  1.0|(692,[158,159,160...|
|  1.0|(692,[124,125,126...|
|  1.0|(692,[152,153,154...|
|  1.0|(692,[151,152,153...|
|  0.0|(692,[129,130,131...|
|  1.0|(692,[158,159,160...|
|  1.0|(692,[99,100,101,...|
|  0.0|(692,[154,155,156...|
|  0.0|(692,[127,128,129...|
|  1.0|(692,[154,155,156...|
|  0.0|(692,[153,154,155...|
|  0.0|(692,[151,152,153...|
|  1.0|(692,[129,130,131...|
|  0.0|(692,[154,155,156...|
|  1.0|(692,[150,151,152...|
|  0.0|(692,[124,125,126...|
|  0.0|(692,[152,153,154...|
|  1.0|(692,[97,98,99,12...|
|  1.0|(692,[124,125,126...|
+-----+--------------------+
only showing top 20 rows



In [5]:
logreg_model = LogisticRegression()

In [6]:
fitted = logreg_model.fit(my_data)

In [8]:
summary = fitted.summary

In [9]:
summary.predictions.printSchema()

root
 |-- label: double (nullable = true)
 |-- features: vector (nullable = true)
 |-- rawPrediction: vector (nullable = true)
 |-- probability: vector (nullable = true)
 |-- prediction: double (nullable = false)



/usr/local/spark/python/pyspark/sql/context.py:125: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [10]:
summary.predictions.show()

/usr/local/spark/python/pyspark/sql/context.py:125: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(692,[127,128,129...|[20.3777627514872...|[0.99999999858729...|       0.0|
|  1.0|(692,[158,159,160...|[-21.114014198868...|[6.76550380000472...|       1.0|
|  1.0|(692,[124,125,126...|[-23.743613234676...|[4.87842678716177...|       1.0|
|  1.0|(692,[152,153,154...|[-19.192574012720...|[4.62137287298144...|       1.0|
|  1.0|(692,[151,152,153...|[-20.125398874699...|[1.81823629113068...|       1.0|
|  0.0|(692,[129,130,131...|[20.4890549504196...|[0.99999999873608...|       0.0|
|  1.0|(692,[158,159,160...|[-21.082940212814...|[6.97903542823766...|       1.0|
|  1.0|(692,[99,100,101,...|[-19.622713503550...|[3.00582577446132...|       1.0|
|  0.0|(692,[154,155,156...|[21.1594863606582...|[0.99999999935352...|       0.0|
|  0.0|(692,[127

In [11]:
train_data, test_data = my_data.randomSplit([0.7, 0.3])

In [12]:
final_model = LogisticRegression()

In [13]:
fit_final = final_model.fit(train_data)

In [14]:
pred_and_labels = fit_final.evaluate(test_data)

In [15]:
pred_and_labels.predictions.show()

+-----+--------------------+--------------------+--------------------+----------+
|label|            features|       rawPrediction|         probability|prediction|
+-----+--------------------+--------------------+--------------------+----------+
|  0.0|(692,[100,101,102...|[3.28766685609126...|[0.96400327938201...|       0.0|
|  0.0|(692,[124,125,126...|[37.1082786524381...|           [1.0,0.0]|       0.0|
|  0.0|(692,[124,125,126...|[21.5179091087729...|[0.99999999954825...|       0.0|
|  0.0|(692,[126,127,128...|[39.1940882671725...|           [1.0,0.0]|       0.0|
|  0.0|(692,[126,127,128...|[19.2526706253945...|[0.99999999564817...|       0.0|
|  0.0|(692,[127,128,129...|[18.7460923381685...|[0.99999999277769...|       0.0|
|  0.0|(692,[127,128,129...|[24.0304330979069...|[0.99999999996338...|       0.0|
|  0.0|(692,[128,129,130...|[24.0954410488584...|[0.99999999996568...|       0.0|
|  0.0|(692,[151,152,153...|[32.0338817291237...|[0.99999999999998...|       0.0|
|  0.0|(692,[153

/usr/local/spark/python/pyspark/sql/context.py:125: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [16]:
from pyspark.ml.evaluation import BinaryClassificationEvaluator, MulticlassClassificationEvaluator

In [17]:
my_eval = BinaryClassificationEvaluator()

In [18]:
roc_result = my_eval.evaluate(pred_and_labels.predictions)

/usr/local/spark/python/pyspark/sql/context.py:125: FutureWarning: Deprecated in 3.0.0. Use SparkSession.builder.getOrCreate() instead.
  warnings.warn(


In [19]:
roc_result

1.0